In [1]:
# Import necessary modules
import sys
sys.path.append("../src/confpickem/")
from yahoo_pickem_scraper import YahooPickEm, calculate_player_stats
from confidence_pickem_sim import ConfidencePickEmSimulator, Player
from yahoo_pickem_integration import convert_yahoo_to_simulator_format, convert_yahoo_picks_to_dataframe
import pandas as pd

In [2]:
# Pull data from Yahoo
week = 10
league_id = 6207
num_sims = 1000
# num_entries = 50

yahoo_data = YahooPickEm(week, league_id, cookies_file='../src/confpickem/cookies.txt')
yahoo_data.games
# yahoo_data.players
# yahoo_data.results

,favorite,favorite_pick_pct,underdog,underdog_pick_pct,home_favorite,favorite_confidence,underdog_confidence,spread,win_prob,kickoff_time
0,Bal,91.0,Cin,9.0,False,8.2,3.8,8.2,0.7542,2024-11-07 20:15:00-05:18
1,NYG,81.0,Car,19.0,True,5.1,2.9,5.1,0.6581,2024-11-10 09:30:00-05:18
2,Chi,93.0,NE,7.0,False,7.2,3.4,7.2,0.7232,2024-11-10 13:00:00-05:18
3,Buf,97.0,Ind,3.0,True,10.4,4.1,10.4,0.8224,2024-11-10 13:00:00-05:18
4,KC,95.0,Den,5.0,False,11.2,3.5,11.2,0.8472,2024-11-10 13:00:00-05:18
5,Atl,94.0,NO,6.0,True,8.1,3.9,8.1,0.7511,2024-11-10 13:00:00-05:18
6,SF,80.0,TB,20.0,True,6.7,3.9,6.7,0.7077,2024-11-10 13:00:00-05:18
7,Was,69.0,Pit,31.0,False,4.9,3.9,4.9,0.6519,2024-11-10 13:00:00-05:18
8,Min,97.0,Jax,3.0,True,10.1,4.4,10.1,0.8131,2024-11-10 13:00:00-05:18
9,LAC,98.0,Ten,2.0,False,10.2,4.9,10.2,0.8162,2024-11-10 16:05:00-05:18


In [3]:
# Initialize simulator instance and feed in NFL game details
simulator = ConfidencePickEmSimulator(num_sims=num_sims)
games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=True)
simulator.add_games_from_dataframe(games_df)

# games_df
# simulator.games

In [4]:
# Create average players and feed into simulator
# simulator.players = [
#         Player(
#             name=f"Player {ind + 1}",
#             skill_level=0.749,
#             crowd_following=0.737,
#             confidence_following=0.826
#         ) for ind in range(num_entries)
#     ]

stats = calculate_player_stats(
        league_id=6207,
        weeks=range(1, week),
        cookies_file='../src/confpickem/cookies.txt'
    )
stats['me'] = stats.player == "Firman's Educated Guesses"
stats = stats.sort_values(by="me",ascending=False,ignore_index=True)

player_stats = [
        Player(
            name=stats.iloc[ind].player,
            skill_level=stats.iloc[ind].skill_level,
            crowd_following=stats.iloc[ind].crowd_following,
            confidence_following=stats.iloc[ind].confidence_following
        ) for ind in range(stats.shape[0])
    ]
simulator.players = player_stats

simulator.players

[Player(name="Firman's Educated Guesses", skill_level=np.float64(0.0), crowd_following=np.float64(0.0), confidence_following=np.float64(0.4179286775339016)),
 Player(name='John K', skill_level=np.float64(0.4944970565651393), crowd_following=np.float64(0.42305334150827706), confidence_following=np.float64(0.8145412977471118)),
 Player(name='California Dreaming', skill_level=np.float64(0.6142820578448941), crowd_following=np.float64(0.7816063764561619), confidence_following=np.float64(0.8327637809701454)),
 Player(name='Matt', skill_level=np.float64(0.6844125927821859), crowd_following=np.float64(0.7144083384426733), confidence_following=np.float64(0.7376610305460428)),
 Player(name='Boats n Hoes', skill_level=np.float64(0.6357819298694654), crowd_following=np.float64(0.8461066830165541), confidence_following=np.float64(0.9187190066586209)),
 Player(name='BigShroomSecurity', skill_level=np.float64(0.3532121832608139), crowd_following=np.float64(0.5415082771305948), confidence_following=n

In [5]:
# Run simulation with random picks and outcomes
picks_df = simulator.simulate_picks()
outcomes = simulator.simulate_outcomes()
# picks_df.groupby(["player","game","picked_home"]).size().sort_values(ascending=False)
# outcomes.sum(axis=0)

# Analyze results
stats_orig = simulator.analyze_results(picks_df, outcomes)
for stat in stats_orig:
    stats_orig[stat] = stats_orig[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})


In [6]:
summary_orig = pd.merge(left=stats_orig["win_pct"],right=stats_orig["expected_points"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["point_std"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["value_at_risk"],how="inner",on="player")
summary_orig

,player,win_pct,expected_points,point_std,value_at_risk
0,#FlytheW,0.021,59.1375,15.556827,32.000
1,9 inches,0.017,56.0705,15.802072,30.000
2,Arcedinero,0.023,56.7000,15.639552,31.000
3,BFense,0.012,54.2300,15.628399,28.000
4,Ballbusters,0.014,55.3950,16.247645,28.000
5,Balz Deep,0.011,55.1145,15.763776,29.000
6,Big Daddy125,0.042,60.3190,15.900694,35.000
7,BigShroomSecurity,0.034,59.9835,15.720404,34.000
8,Boats n Hoes,0.008,54.8670,15.453823,29.000
9,Boldz36,0.019,55.7200,15.673920,30.000


In [7]:
optimal_picks = simulator.optimize_picks()

Game(home_team='Den', away_team='KC', vegas_win_prob=0.8472, crowd_home_pick_pct=0.05, crowd_home_confidence=3.5, crowd_away_confidence=11.2, week=10, kickoff_time=Timestamp('2024-11-10 13:00:00-0518', tz='UTC-05:18'), actual_outcome=None, picks_locked=False)
{'expected_value': np.float64(69.628), 'win_pct': np.float64(0.137)}
{'expected_value': np.float64(58.964), 'win_pct': np.float64(0.044)}
{'Den': 14}
Game(home_team='Buf', away_team='Ind', vegas_win_prob=0.8224, crowd_home_pick_pct=0.97, crowd_home_confidence=10.4, crowd_away_confidence=4.1, week=10, kickoff_time=Timestamp('2024-11-10 13:00:00-0518', tz='UTC-05:18'), actual_outcome=None, picks_locked=False)
{'expected_value': np.float64(77.5765), 'win_pct': np.float64(0.27)}
{'expected_value': np.float64(69.7635), 'win_pct': np.float64(0.115)}
{'Den': 14, 'Buf': 13}
Game(home_team='Ten', away_team='LAC', vegas_win_prob=0.8162, crowd_home_pick_pct=0.02, crowd_home_confidence=4.9, crowd_away_confidence=10.2, week=10, kickoff_time=Ti

In [8]:
# Setting my picks to the optimized values
mask = (picks_df.player == simulator.players[0].name)
for team, pts in optimal_picks.items():
    game_mask = picks_df[mask].game.str.contains(team)
    is_home = (picks_df.loc[mask & game_mask, 'game'].str.split('@').str[1] == team).astype(float)
    picks_df.loc[mask & game_mask, 'picked_home'] = is_home
    picks_df.loc[mask & game_mask, 'confidence'] = float(pts)

# Analyze results
stats_new = simulator.analyze_results(picks_df, outcomes)
for stat in stats_new:
    stats_new[stat] = stats_new[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})

In [9]:
summary_new = pd.merge(left=stats_new["win_pct"],right=stats_new["expected_points"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["point_std"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["value_at_risk"],how="inner",on="player")
summary_new.sort_values(by="win_pct",ascending=False)


,player,win_pct,expected_points,point_std,value_at_risk
0,#FlytheW,0.012,59.1375,15.556827,32.000
1,9 inches,0.015,56.0705,15.802072,30.000
2,Arcedinero,0.020,56.7000,15.639552,31.000
3,BFense,0.010,54.2300,15.628399,28.000
4,Ballbusters,0.011,55.3950,16.247645,28.000
5,Balz Deep,0.008,55.1145,15.763776,29.000
6,Big Daddy125,0.029,60.3190,15.900694,35.000
7,BigShroomSecurity,0.024,59.9835,15.720404,34.000
8,Boats n Hoes,0.005,54.8670,15.453823,29.000
9,Boldz36,0.018,55.7200,15.673920,30.000


### Okay, now try to simulate these picks amongst the actual picks from all the other players!!!
#### Maybe see if any would have won? Or whether this is all bullshit?

In [10]:
picks_df_actual = convert_yahoo_picks_to_dataframe(yahoo_data, num_sims, optimal_picks)
picks_df_actual.loc[picks_df_actual.player == "Firman's Educated Guesses",["game","picked_home"]].drop_duplicates()
# picks_df.picked_home.unique()


,game,picked_home
711000,Bal@Cin,True
712000,Car@NYG,True
713000,Chi@NE,True
714000,Ind@Buf,True
715000,KC@Den,True
716000,NO@Atl,True
717000,TB@SF,True
718000,Was@Pit,True
719000,Jax@Min,True
720000,LAC@Ten,True


In [15]:
outcomes = simulator.simulate_outcomes()
stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
for stat in stats_actual:
    stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
summary_actual.sort_values(by="win_pct",ascending=False)

,player,win_pct,expected_points,point_std,value_at_risk
12,California Dreaming,1.0,91.0,0.0,91.0
0,#FlytheW,0.0,78.0,0.0,78.0
40,Pikachu,0.0,42.0,0.0,42.0
29,MDTwinTeam,0.0,68.0,0.0,68.0
30,MR 16,0.0,73.0,0.0,73.0
31,Matt,0.0,85.0,0.0,85.0
32,Mickey's Team,0.0,68.0,0.0,68.0
33,Mopar,0.0,80.0,0.0,80.0
34,NFL in PINK,0.0,62.0,0.0,62.0
35,NoWayIGotMorePointsThenYou?!?!,0.0,83.0,0.0,83.0


In [12]:
simulator = ConfidencePickEmSimulator(num_sims=num_sims)
games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=False)
simulator.add_games_from_dataframe(games_df)
outcomes = simulator.simulate_outcomes()

stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
for stat in stats_actual:
    stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
summary_actual.sort_values(by="win_pct",ascending=False)

,player,win_pct,expected_points,point_std,value_at_risk
0,#FlytheW,0.0,78.0,0.0,78.0
1,9 inches,0.0,70.0,0.0,70.0
2,Arcedinero,0.0,73.0,0.0,73.0
3,BFense,0.0,67.0,0.0,67.0
4,Ballbusters,0.0,85.0,0.0,85.0
5,Balz Deep,0.0,77.0,0.0,77.0
6,Big Daddy125,0.0,78.0,0.0,78.0
7,BigShroomSecurity,0.0,81.0,0.0,81.0
8,Boats n Hoes,0.0,78.0,0.0,78.0
9,Boldz36,0.0,59.0,0.0,59.0
